# What to do now :

Make a Portfolio using the strategy given. Test the strategy on a historical dataset. Evaluate the performance of the strategy using metrics such as Sharpe Ratio, etc. Make graphical representation.

Note: We are using data for a day.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# No Need to download data again
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
# print(t)
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
tickers=[]
for stock_group in t:
    for stock_name in stock_group:
        tickers.append(stock_name)
    
tickers


## STRATEGIES ------------------------------------------------------------------------ START

In [ ]:
from scipy import stats
                                
def strat_per(stock, max_weight=0.08, momentum_days=20, reversal_days=5):
    """
    Simpler strategy combining short-term reversal with medium-term momentum
    """
    dfs = []
    n = None

    for ticker in stock:
        df = pd.read_csv(f"{ticker}.csv")
        df.reset_index(drop=True, inplace=True)
        df['Close'] = pd.to_numeric(df['Close'])
        dfs.append(df)
        if n is None:
            n = len(df)

    portfolio = np.zeros((len(stock), n))

    for i, df in enumerate(dfs):
        returns = df['Close'].pct_change()
        
        # Medium-term momentum
        momentum = returns.rolling(window=momentum_days).mean()
        
        # Short-term reversal
        short_term_return = returns.rolling(window=reversal_days).mean()
        reversal = -short_term_return  # Opposite of recent performance
        
        # Combine: follow medium-term trend but fade short-term moves
        signal = 0.7 * momentum + 0.3 * reversal # Try changing parameters. Might get some better results 
        
        # Volatility adjustment
        vol = returns.rolling(window=20).std()
        vol_adjusted_signal = signal / (vol + 1e-8) # Avoid division by zero. Important.
        
        portfolio[i, :] = vol_adjusted_signal.shift(1).fillna(0).to_numpy()

    abs_sum = np.sum(np.abs(portfolio), axis=0, keepdims=True)
    abs_sum[abs_sum == 0] = 1.0 # Prevents division by 0
    normalized_portfolio = portfolio / abs_sum # Normalising the portfolio to have a sum of 1
    clipped_portfolio = np.clip(normalized_portfolio, -max_weight, max_weight) # Clip to prevent over-weighting
    market_neutral_portfolio = clipped_portfolio - clipped_portfolio.mean(axis=0, keepdims=True) # Subtract the mean to make it market neutral
    
    return market_neutral_portfolio


## Assignment 
Try creating your own strategies. Just remember, input can be anything ,but , output should be a portfolio with rows as stocks and columns as dates. See example output if you don't understand.
Don't worry if this doesn't correspond to the other backtester. This is because the normalization you might be doing here is different than than how the other does that.

P.S. Before you backtest , make sure to complete the backtester : backtest_portfolio () function. 

In [ ]:
def your_strat_1(stonks):
    return -1

In [ ]:
def your_strat_2(stocks):
    return -1

In [ ]:
def your_strat_3(stocks):
    return -1

## STRATEGIES ------------------------------------------------------ END

In [ ]:
print("Testing the strat......\n")
portfolio = strat_ema_returns(tickers)
# print(portfolio.shape)
print(portfolio)  # Show first 10 elements
print(np.sum(portfolio))   # Sum of allocations : should be ideally 0 after Market Neutralisation

Testing the strat......



C:\Users\91753\AppData\Local\Temp\ipykernel_18836\3180040760.py:122: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df['Close'].pct_change()


[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.32455148e-03
   1.74173052e-03  1.99867923e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  5.58808915e-04
   3.79531055e-04  8.63112333e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.08889048e-03
   2.93352418e-03  3.75828727e-03]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -5.26268926e-05
   9.99455756e-04  1.90593620e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  4.23801635e-04
   9.59684044e-05  1.25555091e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.86073359e-03
   1.89996648e-03  2.94608644e-03]]
-6.994405055138486e-15


In [ ]:
print(portfolio.shape)

(503, 1006)


# Studying the metrics

The strategy was written in such a way that we didn't need to make a portfolio. 

All we had to do is just call the strat and the portfolio would take care of itself.

We have the portfolio, which is what the person's standing for the day is.

How much percentage of his portfolio is each stock in? So, basically we know the no. of stocks he owns for the day, given the portfolio.

So what do we need to do now ?

- As a day passes , we need to update the portfolio. ( We can also assume the portfolio to be constant for simplicity but we won't )

            PORTFOLIO_VAL_NEW  = PORTFOLIO_VAL_OLD + PnL
        
 We can also clip it for saftey.


- Calculate metrics as needed :
    
    - Total PnL

                    PnL = Summ ( ( Close_new- Close_old ) * Quantity ) for all stocks in the portfolio
    
    - Cumilitive Returns

                  Cumulative Return=( ∏ ( 1 + r_t ) ) − 1  : r_t daily return

    - Sharpe Ratio 

                    Sharpe Ratio = ( PnL - Risk_free ) / Volatility

        aka

                    Sharpe Ration = (Mean(Ret_daily) / Std(Ret_daily)) * sqrt(252)

    - Maximum Drawdown

                    Max Drawdown = max { ( V_peak - V_trough ) / V_peak } 
                    V :  portfolio value at time t = Sum ( weight of stock * Price )


    - Turnover

                    Turnover_at_t = Sum (( | weight_new - weight_old | )) : for all stocks in the portfolio

                    Turnover = Sum ( Turnover_at_t ) / days : for all days in the portfolio


    - Correlation : A metric introduced by me. It is the correlation between the stock and the portfolio. We'll measure it in time shift. 

                    Corr(  delta(close*volume,1) , PnL created by Portfolio  )  


- Plotting the metrics : ) 

Note : we have used a lot of assumptions while calculating the metrics and creating this backtester. For example : we have assumed that he player's influence in the market is negligible. 

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as py

def backtest_portfolio(portfolio, tickers, risk_free_rate=0.0451):
    n_stocks, n_days = portfolio.shape
    price_matrix = np.zeros((n_stocks, n_days))

    # Load price data
    for i, ticker in enumerate(tickers):
        df = pd.read_csv(f"{ticker}.csv")
        df = df.reset_index(drop=True)
        df['Close'] = pd.to_numeric(df['Close'])
        price_matrix[i, :] = df['Close'].to_numpy()

    # # Stock daily returns
    stock_returns = pd.DataFrame(price_matrix).pct_change(axis=1).fillna(0).values

    # print(stock_returns)

    # Portfolio returns
    daily_returns = np.nansum(portfolio * stock_returns, axis=0)

    # Cumulative Returns
    cumulative_returns =  .................... Complete this and run

    # Sharpe Ratio
    excess_returns = daily_returns#- risk_free_rate / 252
    sharpe_ratio = ..................... Complete this and run

    # Max Drawdown
    cumulative_curve = np.cumprod(1 + daily_returns)
    rolling_max = np.maximum.accumulate(cumulative_curve)
    drawdown = 1 - cumulative_curve / rolling_max
    max_drawdown = np.max(drawdown)

    # Turnover
    turnover = ...................... Complete this and run
    
    # Mean Correlation with individual stocks
    correlations = []
    for i in range(n_stocks):
        x = daily_returns
        y = stock_returns[i, :]
        mask = ~np.isnan(x) & ~np.isnan(y)
        if np.sum(mask) > 1:
            corr = np.corrcoef(x[mask], y[mask])[0, 1]
            correlations.append(corr)
        else:
            correlations.append(np.nan)
    mean_correlation = np.nanmean(correlations)

    results = {
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "Turnover": turnover,
        "Total Return": cumulative_returns[-1] * 100,
        "Mean Correlation (Portfolio vs Stocks)": mean_correlation,
        "Daily Returns": daily_returns,
        "Cumulative Returns": cumulative_returns
    }

    # PLotting 
    days = np.arange(n_days)

    # Create subplots layout: 3 rows for returns, drawdown, top stocks
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=("Cumulative Returns", "Daily Returns", "Drawdown Curve", "Portfolio vs Top 5 Stocks"),
        vertical_spacing=0.1,
        shared_xaxes=True
    )

    # Cumulative Returns
    fig.add_trace(go.Scatter(x=days, y=cumulative_returns, name="Cumulative Return"), row=1, col=1)

    # Daily Returns
    fig.add_trace(go.Scatter(x=days, y=daily_returns, name="Daily Return"), row=2, col=1)

    # Drawdown
    fig.add_trace(go.Scatter(x=days, y=drawdown, name="Drawdown"), row=3, col=1)

    # Portfolio vs Top 5 correlated stocks
    fig.add_trace(go.Scatter(x=days, y=daily_returns, name="Portfolio"), row=4, col=1)
    top_5_idx = np.argsort(correlations)[-5:]

    # for i in top_5_idx:
    #     print(stock_returns[i])
    # print(top_5_idx)

    for i in top_5_idx:
        fig.add_trace(go.Scatter(x=days, y=stock_returns[i], name=f"{tickers[i]}"), row=4, col=1)

    # Layout
    fig.update_layout(
        height=1200,
        title_text="Portfolio Performance Summary",
        showlegend=True
    )

    # Save and open in browser
    py.plot(fig, filename="portfolio_summary.html")

    return results, correlations


In [ ]:
results,corr = backtest_portfolio(portfolio, tickers)

for key, value in results.items():
    if key not in ['Daily Returns', 'Cumulative Returns']:
        print(f"{key}: {value:.4f}")

# print(corr)

counter =0;
for x in corr:
    if x>0:
        counter+=1

print("No. of correlating stocks : "+ str(counter))

C:\Users\91753\AppData\Local\Temp\ipykernel_18836\847465321.py:21: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\91753\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning:

invalid value encountered in divide

C:\Users\91753\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning:

invalid value encountered in divide



[[ 0.00000000e+00 -1.60561498e-02  2.95654504e-03 ...  7.15954094e-05
  -3.57620493e-04 -3.61202326e-03]
 [ 0.00000000e+00 -9.72164367e-03  7.96826317e-03 ...  5.17765083e-04
   2.22643934e-03 -5.42422955e-03]
 [ 0.00000000e+00 -9.49212766e-03  7.89185558e-03 ...  1.68156657e-03
  -8.39371824e-04  1.42171676e-03]
 ...
 [ 0.00000000e+00 -2.61391420e-03 -5.77913053e-03 ... -2.38840058e-03
   4.12779524e-03  5.75432032e-04]
 [ 0.00000000e+00 -1.19241595e-02  7.65484111e-03 ...  1.05266259e-03
  -1.59542561e-03 -7.33618844e-03]
 [ 0.00000000e+00  1.48919903e-04 -7.67726435e-03 ...  7.16111339e-03
   1.32055958e-03  1.06505998e-03]]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[201 414  67  57 423]
Sharpe Ratio: 0.1387
Max Drawdown: 0.0999
Turnover: 0.4226
Total Return: 2.5677
Mean Correlation (Portfolio vs Stocks): -0.0649
No. of correlating stocks : 44
